# Gallery Example: M/M/1 Reentrant Network

This example demonstrates an M/M/1 reentrant network where jobs:
- Visit the same station multiple times
- Follow a specific reentrant routing pattern
- Create complex dependencies and potential bottlenecks

Reentrant networks are common in manufacturing and computer systems where jobs require multiple passes through the same resources.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_reentrant(): Network {    """Create M/M/1 reentrant network"""    model = Network("M/M/1-Reentrant")        # Block 1: nodes    source = Source(model, "Source")    queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)    queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)     sink = Sink(model, "Sink")        # Block 2: classes - multiple classes for different routing stages    class1 = OpenClass(model, "Stage1")  # First visit to Queue1    class2 = OpenClass(model, "Stage2")  # Visit to Queue2    class3 = OpenClass(model, "Stage3")  # Second visit to Queue1 (reentrant)        # Arrivals only to first class    source.setArrival(class1, Exp(0.5))  # External arrivals    source.setArrival(class2, Exp(0))    # No direct arrivals    source.setArrival(class3, Exp(0))    # No direct arrivals        # Service rates - same server, different service requirements per stage    queue1.setService(class1, Exp(4))    # First visit: fast service    queue1.setService(class2, Exp(0))    # No service for class2 at queue1    queue1.setService(class3, Exp(2))    # Reentrant visit: slower service        queue2.setService(class1, Exp(0))    # No service for class1 at queue2    queue2.setService(class2, Exp(3))    # Service for class2    queue2.setService(class3, Exp(0))    # No service for class3 at queue2        # Block 3: reentrant routing    P = model.initRoutingMatrix()        # Stage 1: Source -> Queue1 (as class1) -> Queue2 (becomes class2)    P.addRoute(class1, source, queue1, 1.0)    P.addRoute(class1, queue1, queue2, 1.0)  # Class switching happens here        # Stage 2: Queue2 (as class2) -> Queue1 (becomes class3, reentrant)    P.addRoute(class2, queue2, queue1, 1.0)  # Class switching happens here        # Stage 3: Queue1 (as class3) -> Sink (exit)    P.addRoute(class3, queue1, sink, 1.0)        model.link(P)        return model# Create the modelmodel = gallery_mm1_reentrant()println(f"\nReentrant pattern: Queue1 -> Queue2 -> Queue1 (reentry) -> Exit")

## Reentrant Network Analysis

In this reentrant network:
- **Queue1** serves jobs twice (stages 1 and 3)
- **Queue2** serves jobs once (stage 2)
- **Load multiplication**: Queue1 handles 2x the external arrival rate
- **Bottleneck effects**: Queue1 may become heavily loaded

Total load on Queue1 = λ (stage 1) + λ (stage 3) = 2λ
Total load on Queue2 = λ (stage 2)

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
try:
    val solver_mva = MVA(model)
    val avg_table_mva = solver_mva.avgTable
    println("\nMVA Solver:")
    println(avg_table_mva)
except Exception as e:
    println(f"MVA Solver error: {e}")
// CTMC Solver
try:
    val solver_ctmc = CTMC(model, "cutoff", 10)
    val avg_table_ctmc = solver_ctmc.avgTable
    println("\nCTMC Solver:")
    println(avg_table_ctmc)
except Exception as e:
    println(f"CTMC Solver error: {e}")
// Fluid Solver
try:
    val solver_fluid = FLD(model)
    val avg_table_fluid = solver_fluid.avgTable
    println("\nFluid Solver:")
    println(avg_table_fluid)
except Exception as e:
    println(f"Fluid Solver error: {e}")

In [ ]:
// Alternative simpler reentrant model using class switching
println("\n=== Alternative Reentrant Model (Class Switching) ===")

fun create_reentrant_with_class_switch(): Network {
    """Create reentrant network using explicit class switching"""
    val model_cs = Network("Reentrant-ClassSwitch")
// Nodes
    val source = Source(model_cs, "Source")
    val queue1 = Queue(model_cs, "Queue1", SchedStrategy.FCFS)
    val queue2 = Queue(model_cs, "Queue2", SchedStrategy.FCFS)
    val sink = Sink(model_cs, "Sink")
// Single class with class switching routing
    val oclass = OpenClass(model_cs, "Jobs")
    
    source.setArrival(oclass, Exp(0.5))
    queue1.setService(oclass, Exp(3))  # Average of both visits
    queue2.setService(oclass, Exp(3))
// Routing with feedback to create reentrant behavior
    val P = model_cs.initRoutingMatrix()
    P.addRoute(oclass, source, queue1, 1.0)
    P.addRoute(oclass, queue1, queue2, 0.8)  # Most go to queue2
    P.addRoute(oclass, queue1, sink, 0.2)    # Some exit (reentrant jobs)
    P.addRoute(oclass, queue2, queue1, 1.0)  # All return to queue1 (reentry)
    
    model_cs.link(P)
    return model_cs

val model_cs = create_reentrant_with_class_switch()
val solver_cs = MVA(model_cs)
val avg_table_cs = solver_cs.avgTable

println("Class Switching Reentrant Network:")
println(avg_table_cs)
// Analyze effective load multiplication
val queue1_util = float(avg_table_cs.iloc[1, 1])  # Queue1 utilization
val queue2_util = float(avg_table_cs.iloc[2, 1])  # Queue2 utilization

println(f"\nLoad Analysis:")
println(f"Queue1 utilization: {queue1_util:.3f} (handles multiple visits)")
println(f"Queue2 utilization: {queue2_util:.3f} (handles single visits)")
println(f"Load ratio Q1/Q2: {queue1_util/queue2_util:.2f}" if queue2_util > 0 else "Queue2 has no load")

In [ ]:
// Compare with equivalent tandem network (no reentrancy)
println("\n=== Comparison with Non-Reentrant Tandem ===")

fun create_equivalent_tandem(): Network {
    """Create equivalent tandem network for comparison"""
    val model_tandem = Network("Equivalent-Tandem")
    
    val source = Source(model_tandem, "Source")
    val queue1 = Queue(model_tandem, "Queue1", SchedStrategy.FCFS)
    val queue2 = Queue(model_tandem, "Queue2", SchedStrategy.FCFS)
    val sink = Sink(model_tandem, "Sink")
    
    val oclass = OpenClass(model_tandem, "Jobs")
    source.setArrival(oclass, Exp(0.5))
    queue1.setService(oclass, Exp(3))  # Same service rate
    queue2.setService(oclass, Exp(3))  # Same service rate
// Simple tandem routing
    val P = model_tandem.initRoutingMatrix()
    P.addRoute(oclass, source, queue1, 1.0)
    P.addRoute(oclass, queue1, queue2, 1.0)
    P.addRoute(oclass, queue2, sink, 1.0)
    
    model_tandem.link(P)
    return model_tandem

val model_tandem = create_equivalent_tandem()
val solver_tandem = MVA(model_tandem)
val avg_table_tandem = solver_tandem.avgTable

println("Tandem Network (no reentrancy):")
println(avg_table_tandem)
// Compare performance
val tandem_q1_util = float(avg_table_tandem.iloc[1, 1])
val tandem_q2_util = float(avg_table_tandem.iloc[2, 1])
val tandem_total_resp = float(avg_table_tandem.iloc[1, 2]) + float(avg_table_tandem.iloc[2, 2])

val cs_total_resp = float(avg_table_cs.iloc[1, 2]) + float(avg_table_cs.iloc[2, 2])

println(f"\nPerformance Comparison:")
println(f"Tandem Network:")
println(f"  Q1 utilization: {tandem_q1_util:.3f}")
println(f"  Q2 utilization: {tandem_q2_util:.3f}")
println(f"  Total response: {tandem_total_resp:.3f}")

println(f"\nReentrant Network:")
println(f"  Q1 utilization: {queue1_util:.3f} (higher due to reentrancy)")
println(f"  Q2 utilization: {queue2_util:.3f}")
println(f"  Total response: {cs_total_resp:.3f}")

println(f"\nReentrancy Impact: {(cs_total_resp / tandem_total_resp):.2f}x response time increase")
println("Note: Reentrancy creates additional load and contention at revisited stations.")